In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Defining file path
ecg_folder = "../../../../Datasets/12-lead electrocardiogram database/ECGDataDenoised"
attributes_file = "../../../../Datasets/12-lead electrocardiogram database/AttributesDictionary.xlsx"
diagnostics_file = "../../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"
rhythm_names_file = "../../../../Datasets/12-lead electrocardiogram database/RhythmNames.xlsx"

# Checking for missing files and stuff
for file_path in [attributes_file, diagnostics_file, rhythm_names_file]:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Required file not found: {file_path}")

# Loading metadata
attributes_df = pd.read_excel(attributes_file)
diagnostics_df = pd.read_excel(diagnostics_file)
rhythm_names_df = pd.read_excel(rhythm_names_file)

# Removing trailing spaces in acronym columns for accurate matching
rhythm_names_df['Acronym Name'] = rhythm_names_df['Acronym Name'].str.strip()

# Creating sets of valid acronyms for rhythm
valid_rhythms = set(rhythm_names_df['Acronym Name'])
print(valid_rhythms)

{'SR', 'AFIB', 'AF', 'AVRT', 'SVT', 'SI', 'ST', 'SB', 'AT', 'AVNRT', 'SAAWR'}


In [2]:
# Load ECG data
def load_ecg_data(ecg_folder, diagnostics_df):
    data = []
    labels = []
    metadata = []

    # Check if the folder exists
    if not os.path.exists(ecg_folder):
        raise FileNotFoundError(f"ECG data folder not found: {ecg_folder}")

    for file_name in os.listdir(ecg_folder):
        if file_name.endswith('.csv'):
            # Read ECG data
            ecg_data = pd.read_csv(os.path.join(ecg_folder, file_name), header=None)
            ecg_data = ecg_data.to_numpy()  # Convert to numpy array

            # Retrieve associated label and metadata
            record_id = file_name.replace('.csv', '')
            record_info = diagnostics_df[diagnostics_df['FileName'] == record_id]

            # Skip if no label information is available
            if record_info.empty:
                raise ValueError(f"No diagnostic information found for file: {file_name}")

            # Get rhythm and condition acronyms directly
            rhythm_acronym = record_info['Rhythm'].values[0].strip()

            # The author of the dataset seems like did an oopsie
            if rhythm_acronym == "SA":
                rhythm_acronym = "SI"
                
            # Validate rhythm and condition acronyms
            if rhythm_acronym not in valid_rhythms:
                raise ValueError(f"Unknown rhythm acronym '{rhythm_acronym}' found in file: {file_name}")

            # Set label as rhythm acronym (or use condition acronym if preferred)
            label = rhythm_acronym  # Use rhythm acronym as label

            # Additional metadata
            patient_age = record_info["PatientAge"].values[0]
            ventricular_rate = record_info["VentricularRate"].values[0]
            atrial_rate = record_info["AtrialRate"].values[0]

            data.append(ecg_data)
            labels.append(label)
            metadata.append({
                "patient_age": patient_age,
                "ventricular_rate": ventricular_rate,
                "atrial_rate": atrial_rate
            })

    return data, np.array(labels), metadata


# Load ECG data and labels
ecg_data, ecg_labels, ecg_metadata = load_ecg_data(ecg_folder, diagnostics_df)

# Standardize each ECG signal individually
scaler = StandardScaler()
ecg_data = [scaler.fit_transform(sample) for sample in ecg_data]

# Define a fixed sequence length (pad or truncate to this length)
sequence_length = 5000  # Adjust as per dataset requirements


# Function to pad or truncate each ECG signal to a fixed length
def preprocess_sequence(data, length):
    processed_data = []
    for sample in data:
        if sample.shape[0] > length:
            processed_data.append(sample[:length])
        else:
            pad_width = length - sample.shape[0]
            processed_data.append(np.pad(sample, ((0, pad_width), (0, 0)), mode="constant"))
    return np.array(processed_data)


# initiating
ecg_data = preprocess_sequence(ecg_data, sequence_length)

In [3]:
from collections import Counter

# Filter out classes with fewer than 2 instances
label_counts = Counter(ecg_labels)
print(label_counts)
filtered_indices = [i for i, label in enumerate(ecg_labels) if label_counts[label] > 1]
filtered_ecg_data = ecg_data[filtered_indices]
filtered_ecg_labels = ecg_labels[filtered_indices]

# Train-test split with filtered data
X_train, X_test, y_train, y_test = train_test_split(filtered_ecg_data, filtered_ecg_labels, test_size=0.25,
                                                    random_state=42, stratify=filtered_ecg_labels)

# Verify shapes and data summary
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)
print("Training labels shape:", y_train.shape)
print("Testing labels shape:", y_test.shape)
print("Metadata example:", ecg_metadata[0])

Counter({'SB': 3889, 'SR': 1826, 'AFIB': 1780, 'ST': 1568, 'SVT': 587, 'AF': 445, 'SI': 399, 'AT': 121, 'AVNRT': 16, 'AVRT': 8, 'SAAWR': 7})
Training data shape: (7984, 5000, 12)
Testing data shape: (2662, 5000, 12)
Training labels shape: (7984,)
Testing labels shape: (2662,)
Metadata example: {'patient_age': 82, 'ventricular_rate': 143, 'atrial_rate': 144}


In [4]:
X_train

array([[[-4.99469302e-01, -3.46706602e-01,  5.71870791e-02, ...,
         -9.89982759e-02, -1.52578121e-01,  1.07958090e-01],
        [-4.60453887e-01, -3.47442777e-01, -6.97010727e-02, ...,
         -1.38145379e-01, -1.62969205e-01,  2.26572811e-02],
        [-4.10746611e-01, -3.51776342e-01, -2.45500077e-01, ...,
         -1.77558083e-01, -1.78843441e-01, -5.76279899e-02],
        ...,
        [-1.22298004e+00, -9.18835855e-01, -3.20261691e-02, ...,
         -1.08560774e+00, -9.05405961e-01, -1.63877634e+00],
        [-1.37052206e+00, -9.19784534e-01,  4.37098219e-01, ...,
         -1.09644289e+00, -8.99777457e-01, -1.61553181e+00],
        [-1.49953906e+00, -9.20353741e-01,  8.47959096e-01, ...,
         -1.11627954e+00, -9.02560783e-01, -1.60130646e+00]],

       [[-1.07242928e+00, -1.36547729e+00, -5.12857656e-01, ...,
         -2.22476715e-01, -6.14322396e-01, -7.18602923e-01],
        [-9.59694553e-01, -1.36614529e+00, -6.47993617e-01, ...,
         -1.82431931e-01, -5.48641902e

In [5]:
y_train

array(['SB', 'AFIB', 'AFIB', ..., 'AFIB', 'SB', 'SR'], dtype='<U5')

In [6]:
import os

# Directory to save the train-test split data
output_dir = "../../../Datasets/12-lead electrocardiogram database/FirstRevFull5000Sample"

# Creating directory if it doesn't exist already
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# function to save the stuff
def save_split(data, labels, split_name):
    data_file = os.path.join(output_dir, f"X_{split_name}.txt")
    labels_file = os.path.join(output_dir, f"y_{split_name}.txt")

    # Saving the data
    with open(data_file, "w") as f_data:
        for sample in data:
            # Convert each sample to a space-separated string and write to file
            sample_str = " ".join(map(str, sample.flatten()))
            f_data.write(sample_str + "\n")

    # Save labels
    with open(labels_file, "w") as f_labels:
        for label in labels:
            f_labels.write(str(label) + "\n")

# Initialing the process
save_split(X_train, y_train, "train")
save_split(X_test, y_test, "test")

print(f"Train and test data have been saved to the '{output_dir}' directory.")

Train and test data have been saved to the '../../../Datasets/12-lead electrocardiogram database/FirstRevFull5000Sample' directory.
